In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from pprint import pprint

In [2]:
import re
from bs4 import BeautifulSoup
import requests
import pandas as pd

response = requests.get('https://www.ikejaelectric.com/cnn/')
soup = BeautifulSoup(response.content, 'html.parser')

fault_entries = soup.find_all('div', class_='post')

# Lists to store the extracted information
post_categories = []
post_dates = []
undertakings = []
faults = []
areas_affected_s = []

for entry in fault_entries:
    post_category = entry.find('a', class_='post-category').text.strip()
    post_date = entry.find('span', class_='post-date').text.strip()

    # Extract information from the h3 tag
    h3_tag = entry.find('h3', class_='post-title')

    # Extract text from the h3 tag
    h3_text = h3_tag.get_text(separator='\n', strip=True)

    # Extract undertaking, fault, and areas affected using regular expressions
    undertaking_match = re.search(r'UNDERTAKING:(.*?)FAULT:', h3_text, re.DOTALL)
    fault_match = re.search(r'FAULT:(.*?)AREAS AFFECTED:', h3_text, re.DOTALL)
    areas_affected_match = re.search(r'AREAS AFFECTED:(.*?)$', h3_text, re.DOTALL)

    # Check if matches are found before extracting
    undertaking = undertaking_match.group(1).strip() if undertaking_match else "Not found"
    fault = fault_match.group(1).strip() if fault_match else "Not found"
    areas_affected = areas_affected_match.group(1).strip() if areas_affected_match else "Not found"

    # Append the extracted information to the lists
    post_categories.append(post_category)
    post_dates.append(post_date)
    undertakings.append(undertaking)
    faults.append(fault)
    areas_affected_s.append(areas_affected)

# Create a pandas DataFrame from the lists
data = {
    'Branch': post_categories,
    'Date': post_dates,
    'Undertaking': undertakings,
    'Fault': faults,
    'Areas': areas_affected_s
}

df = pd.DataFrame(data)

In [3]:
df = pd.DataFrame(data)
df.head(3)

,Branch,Date,Undertaking,Fault,Areas
0,IKORODU,"Thu, 04 Apr 2024",EPE,AGBOWA 33KV FEEDER OUT ON FAULTS,AGBOWA 33KV FEEDER
1,SHOMOLU,"Thu, 04 Apr 2024",IGBOBI,PROLONGED OUTAGE ON 33-OWORONSHOKITCN-IGBOBI F...,33-OWORONSHOKITCN-IGBOBI FEEDER
2,ABULE-EGBA,"Tue, 02 Apr 2024",IJU,OUTAGE ON ISHAGA 11KV FEEDER,11-IJUINJ-T2-ISHAGA


In [4]:
df['Date'] = pd.to_datetime(df['Date'])

In [6]:
df

,Branch,Date,Undertaking,Fault,Areas
0,ABULE-EGBA,2024-03-01,IJU,CNN NOTIFICATIONS TO CUSTOMERS ON 11-OBAWOLEIN...,CUSTOMERS ON 11-OBAWOLEINJ-T1-SHONUBI-CAJE 2 3...
1,ABULE-EGBA,2024-01-08,IJU,CNN REQUEST TO CUSTOMERS OUT DUE TO FAULT.,11-AGEGEINJ-T1-IJU ROAD-1 OPA ARO CL HVDS – OP...
2,ABULE-EGBA,2024-01-08,IJU,OUTAGE ON GALILEE 11KV FDR,GALILEE 11KV
3,AKOWONJO,2024-01-05,OKE,PROLONG OUTAGE ON OKE ODO 11KV FEEDER,OKE ODO
4,ABULE-EGBA,2023-12-23,IJU,OUTAGE ON GALILEE 11KV FDR,GALILEE 11KV
...,...,...,...,...,...
573,ABULE-EGBA,2021-10-10,FAGBA,FAULT ON AGBE ROAD 11KV FEEDER,CUSTOMERS ON AGBE ROAD 11KV FEEDER
574,ABULE-EGBA,2021-10-10,FAGBA,FAULT ON AGBE ROAD 11KV FEEDER,CUSTOMERS ON AGBE ROAD 11KV FEEDER
575,ABULE-EGBA,2021-10-04,FAGBA,FAULT ON 11-IJAIYE OJOKOROINJ-T1-AGBADO 2-MAJE...,CUSTOMERS ON 11-IJAIYE OJOKOROINJ-T1-AGBADO 2-...
576,SHOMOLU,2021-10-02,OWORO,FAULT ON OWORO IGBOBI 33KV,CUSTOMERS ON OWORO IGBOBI 33KV


In [35]:
#app = Nominatim(user_agent="project")
for index, row in df.iterrows():
    # Access row data
    branch = row['Branch']
    try:
        app = Nominatim(user_agent="project")
        location = app.geocode(branch + " Lagos").raw
    except:
        app = Nominatim(user_agent="project")
        location = app.geocode(branch).raw
    y = location['lat']
    x = location['lon']
    df.at[index, 'y'] = y
    df.at[index, 'x'] = x

In [38]:
df

,Branch,Date,Undertaking,Fault,Areas,y,x
0,ABULE-EGBA,2024-03-01,IJU,CNN NOTIFICATIONS TO CUSTOMERS ON 11-OBAWOLEIN...,CUSTOMERS ON 11-OBAWOLEINJ-T1-SHONUBI-CAJE 2 3...,6.6712,3.3515
1,ABULE-EGBA,2024-01-08,IJU,CNN REQUEST TO CUSTOMERS OUT DUE TO FAULT.,11-AGEGEINJ-T1-IJU ROAD-1 OPA ARO CL HVDS – OP...,6.6712,3.3515
2,ABULE-EGBA,2024-01-08,IJU,OUTAGE ON GALILEE 11KV FDR,GALILEE 11KV,6.6712,3.3515
3,AKOWONJO,2024-01-05,OKE,PROLONG OUTAGE ON OKE ODO 11KV FEEDER,OKE ODO,6.6263,3.2835
4,ABULE-EGBA,2023-12-23,IJU,OUTAGE ON GALILEE 11KV FDR,GALILEE 11KV,6.6712,3.3515
...,...,...,...,...,...,...,...
573,ABULE-EGBA,2021-10-10,FAGBA,FAULT ON AGBE ROAD 11KV FEEDER,CUSTOMERS ON AGBE ROAD 11KV FEEDER,NaN,NaN
574,ABULE-EGBA,2021-10-10,FAGBA,FAULT ON AGBE ROAD 11KV FEEDER,CUSTOMERS ON AGBE ROAD 11KV FEEDER,NaN,NaN
575,ABULE-EGBA,2021-10-04,FAGBA,FAULT ON 11-IJAIYE OJOKOROINJ-T1-AGBADO 2-MAJE...,CUSTOMERS ON 11-IJAIYE OJOKOROINJ-T1-AGBADO 2-...,NaN,NaN
576,SHOMOLU,2021-10-02,OWORO,FAULT ON OWORO IGBOBI 33KV,CUSTOMERS ON OWORO IGBOBI 33KV,NaN,NaN


In [5]:
locations = {'ABULE-EGBA':[6.6478943,3.3046224], 'AKOWONJO':[6.6104787,3.3103079],
             'IKORODU':[6.6191233,3.5041271], 'IKEJA':[6.5960605, 3.340787],
             'OSHODI':[6.5400100000000005,3.3124146026491035], 'SHOMOLU':[6.533564500000001,3.38416340600858]}

In [6]:
df['Undertaking'].unique()

array(['EPE', 'IGBOBI', 'IJU', 'ODOGUNYAN', 'DOPEMU', 'FAGBA', 'AMUWO',
       'MAGODO', 'OKE', 'AYANGBURIN', 'OREGUN', 'EGBEDA', 'PTC', 'AIT',
       'ABULE', 'IGANDO', 'ADIYAN', 'LASUNWON', 'IJAIYE', 'IKOTUN',
       'OWUTU', 'IFAKO', 'OBA', 'OKEIRA', 'OLAMBE', 'AKUTE', 'KETU',
       'MENDE', 'OJODU', 'ILUPEJU', 'OSHODI', 'ISOLO', 'ANIFOWOSHE',
       'BARIGA', 'OLATEJU', 'IDIMU', 'AGO', 'AYOBO', 'OWORO', 'IKOSI',
       'AJAO', 'IPAJA', 'IJEDE', 'IGBOGBO', 'OKOTA', 'GOWON-ESTATE',
       'OLOWORA', 'OGBA', 'IJEGUN', 'OGUDU', 'ABORU', 'ORILE-AGEGE'],
      dtype=object)

In [39]:
df[df['Undertaking'].str.contains('abule', case=False)].head(10)

,Branch,Date,Undertaking,Fault,Areas,y,x
14,AKOWONJO,2023-12-02,ABULE,CNN TO ALL CUSTOMERS FEEDING FROM ALIMOSHO 3X1...,11-ALIMOSHOINJ-T4-AKOWONJO 11-ALIMOSHOINJ-T4-F...,NaN,NaN
31,AKOWONJO,2023-11-13,ABULE,PROLONG OUTAGE ON EKORO 11KV FEEDER,EKORO 11KV FEEDER,NaN,NaN
52,AKOWONJO,2023-10-04,ABULE,CNN TO AFFECTED CUSTOMERS ON AKOWONJO 11KV FEEDER,AKOWONJO 11KV FEEDER,NaN,NaN
58,AKOWONJO,2023-10-03,ABULE,BROADCAST TO CUSTOMERS UNDER 11-EKOROINJ-T2-OL...,11-EKOROINJ-T2-OLOTA-ISE OLUWA 1,NaN,NaN
59,AKOWONJO,2023-08-31,ABULE,BROADCAST TO CUSTOMERS UNDER 11-EKOROINJ-T2-OL...,11-EKOROINJ-T2-OLOTA-ISE OLUWA 1,NaN,NaN
231,AKOWONJO,2022-12-29,ABULE,TCN BREAKER FAULT ON T8 ALIMOSHO,"ALIMOSHO, ABULE ODU, GOWON",NaN,NaN
236,AKOWONJO,2022-12-24,ABULE,TERMINATION FAILURE AT EJIGBO TX,TERMINATION FAILURE AT EJIGBO TX,NaN,NaN
243,AKOWONJO,2022-12-21,ABULE,PLANNED OUTAGE ON ABULE-TAYLOR 33KV FEEDER,ABULE-TAYLOR 33KV FEEDER,NaN,NaN
306,AKOWONJO,2022-10-07,ABULE,PLANNED OUTAGE REQUEST ON ABULE TAYLOR 33KV FE...,ABULE TAYLOR 33KV FEEDER,NaN,NaN
307,AKOWONJO,2022-10-06,ABULE,EMERGENCY OUTAGE ON T2 100MVA ALIMOSHO,EMERGENCY OUTAGE ON T2 100MVA ALIMOSHO,NaN,NaN


In [7]:
zones = {'IJU':[6.6712,3.3515], 'OKE':[6.6263,3.2835], 'AYANGBURIN':[6.619, 3.502],
         'OREGUN':[6.616146, 3.362653], 'EGBEDA':[6.5916, 3.2911], 'FAGBA':[6.657, 3.324],
         'PTC':[6.5920, 3.3435], 'AIT':[6.668, 3.263], 'ABULE': [6.590, 3.285], 'IGANDO':[6.546, 3.248],
         'ADIYAN':[6.69083,3.33972], 'AMUWO':[6.463,3.298], 'LASUNWON':[6.649, 3.514],
         'IJAIYE':[6.629, 3.338], 'IKOTUN':[6.554,3.269], 'OWUTU': [6.645, 3.479], 'IFAKO':[6.683, 3.294],
         'OBA':[6.611, 3.340], 'ODOGUNYAN':[6.673, 3.516], 'OKEIRA':[6.491, 3.370], 'OLAMBE':[6.694, 3.341],
         'AKUTE':[6.682, 3.356],'KETU':[6.598, 3.390],'MENDE':[6.572, 3.372], 'OJODU':[6.634, 3.356],
         'IGBOBI':[6.528, 3.374], 'ILUPEJU':[6.554, 3.356], 'MAGODO':[6.618, 3.400], 'DOPEMU':[6.613, 3.314],
         'OSHODI':[6.554, 3.337], 'ISOLO':[6.538, 3.323], 'ANIFOWOSHE':[6.598, 3.337], 'BARIGA':[6.539, 3.386],
         'OLATEJU':[6.535, 3.359], 'IDIMU':[6.584, 3.246], 'AGO':[6.508, 3.309], 'AYOBO':[6.606, 3.244],
         'OWORO':[6.544, 3.399], 'IKOSI':[6.600, 3.383], 'AJAO':[6.546, 3.326], 'IPAJA':[6.613, 3.266],
         'IJEDE':[6.571, 3.597], 'IGBOGBO':[6.585, 3.525], 'OKOTA':[6.500, 3.310],'GOWON-ESTATE':[6.604, 3.285],
         'OLOWORA':[6.635, 3.384], 'OGBA':[6.625, 3.344], 'IJEGUN': [6.517, 3.262], 'OGUDU':[6.574, 3.394],
         'ABORU':[6.636, 3.283], 'ORILE-AGEGE':[6.629, 3.311], 'EPE':[6.649, 3.713]
        }

In [8]:
#app = Nominatim(user_agent="project")
for index, row in df.iterrows():
    # Access row data
    undertaking = row['Undertaking']
    y = zones[undertaking][0]
    #y = location['lat']
    #x = location['lon']
    x = zones[undertaking][1]
    df.at[index, 'y'] = y
    df.at[index, 'x'] = x

In [9]:
# Calculate the bounding coordinate box
min_x = df['x'].min()
max_x = df['x'].max()
min_y = df['y'].min()
max_y = df['y'].max()

# Create a list to store the bounding coordinate box values
bbox = [min_x, max_x, min_y, max_y]

# Print the bounding coordinate box list
print("Bounding Coordinate Box List:")
print(bbox)

Bounding Coordinate Box List:
[3.244, 3.713, 6.463, 6.694]


In [10]:
gdf = gpd.GeoDataFrame(df, crs='epsg:4326', geometry=gpd.points_from_xy(df.x, df.y))
gdf['Date'] = gdf['Date'].astype(str)
gdf = gdf.cx[bbox[0]:bbox[2], bbox[1]:bbox[3]]

In [11]:
gdf

,Branch,Date,Undertaking,Fault,Areas,y,x,geometry
0,IKORODU,2024-04-04,EPE,AGBOWA 33KV FEEDER OUT ON FAULTS,AGBOWA 33KV FEEDER,6.6490,3.7130,POINT (3.71300 6.64900)
1,SHOMOLU,2024-04-04,IGBOBI,PROLONGED OUTAGE ON 33-OWORONSHOKITCN-IGBOBI F...,33-OWORONSHOKITCN-IGBOBI FEEDER,6.5280,3.3740,POINT (3.37400 6.52800)
2,ABULE-EGBA,2024-04-02,IJU,OUTAGE ON ISHAGA 11KV FEEDER,11-IJUINJ-T2-ISHAGA,6.6712,3.3515,POINT (3.35150 6.67120)
3,IKORODU,2024-04-02,ODOGUNYAN,11-ODOGUNYANINJ-T1-AGODO IS OUT ON FAULT DUE T...,11-ODOGUNYANINJ-T1-AGODO,6.6730,3.5160,POINT (3.51600 6.67300)
4,AKOWONJO,2024-04-02,DOPEMU,PROLONGED OUTAGE PEN CINEMA 11KV FEEDER,11-AGEGEINJ-T3-PEN CINEMA,6.6130,3.3140,POINT (3.31400 6.61300)
...,...,...,...,...,...,...,...,...
579,ABULE-EGBA,2021-11-04,FAGBA,FAULT AFFECTING AGBE RD 11KV FEEDER,FAGBA,6.6570,3.3240,POINT (3.32400 6.65700)
580,ABULE-EGBA,2021-11-04,FAGBA,FAULT AFFECTING AGBE RD 11KV FEEDER,FAGBA,6.6570,3.3240,POINT (3.32400 6.65700)
581,SHOMOLU,2021-10-24,KETU,FAULT ON KETU 11KV FEEDER,CUSTOMERS ON KETU 11KV FEEDER,6.5980,3.3900,POINT (3.39000 6.59800)
582,ABULE-EGBA,2021-10-10,FAGBA,FAULT ON AGBE ROAD 11KV FEEDER,CUSTOMERS ON AGBE ROAD 11KV FEEDER,6.6570,3.3240,POINT (3.32400 6.65700)


In [16]:
gdf.explore()

In [13]:
gdf['y'] = gdf['y'].astype(float)
gdf['x'] = gdf['x'].astype(float)

In [14]:
gdf.to_file('gdf.geojson', driver='GeoJSON') 

In [54]:
gdf

,Branch,Date,Undertaking,Fault,Areas,y,x,geometry
0,ABULE-EGBA,2024-03-01,IJU,CNN NOTIFICATIONS TO CUSTOMERS ON 11-OBAWOLEIN...,CUSTOMERS ON 11-OBAWOLEINJ-T1-SHONUBI-CAJE 2 3...,6.6712,3.3515,POINT (3.35150 6.67120)
1,ABULE-EGBA,2024-01-08,IJU,CNN REQUEST TO CUSTOMERS OUT DUE TO FAULT.,11-AGEGEINJ-T1-IJU ROAD-1 OPA ARO CL HVDS – OP...,6.6712,3.3515,POINT (3.35150 6.67120)
2,ABULE-EGBA,2024-01-08,IJU,OUTAGE ON GALILEE 11KV FDR,GALILEE 11KV,6.6712,3.3515,POINT (3.35150 6.67120)
3,AKOWONJO,2024-01-05,OKE,PROLONG OUTAGE ON OKE ODO 11KV FEEDER,OKE ODO,6.6263,3.2835,POINT (3.28350 6.62630)
4,ABULE-EGBA,2023-12-23,IJU,OUTAGE ON GALILEE 11KV FDR,GALILEE 11KV,6.6712,3.3515,POINT (3.35150 6.67120)
...,...,...,...,...,...,...,...,...
573,ABULE-EGBA,2021-10-10,FAGBA,FAULT ON AGBE ROAD 11KV FEEDER,CUSTOMERS ON AGBE ROAD 11KV FEEDER,6.6570,3.3240,POINT (3.32400 6.65700)
574,ABULE-EGBA,2021-10-10,FAGBA,FAULT ON AGBE ROAD 11KV FEEDER,CUSTOMERS ON AGBE ROAD 11KV FEEDER,6.6570,3.3240,POINT (3.32400 6.65700)
575,ABULE-EGBA,2021-10-04,FAGBA,FAULT ON 11-IJAIYE OJOKOROINJ-T1-AGBADO 2-MAJE...,CUSTOMERS ON 11-IJAIYE OJOKOROINJ-T1-AGBADO 2-...,6.6570,3.3240,POINT (3.32400 6.65700)
576,SHOMOLU,2021-10-02,OWORO,FAULT ON OWORO IGBOBI 33KV,CUSTOMERS ON OWORO IGBOBI 33KV,6.5440,3.3990,POINT (3.39900 6.54400)


In [17]:
import folium
from folium.plugins import HeatMap

# Create a map centered at a location
m = folium.Map(location=[gdf['y'].mean(), gdf['x'].mean()], zoom_start=10)

# Convert the GeoDataFrame to a list of points
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in gdf['geometry']]

# Add the HeatMap layer to the map
HeatMap(heat_data).add_to(m)

m

In [18]:
from folium import plugins
from folium.plugins import MarkerCluster
# Create a map centered at a location
m = folium.Map(location=[gdf['y'].mean(), gdf['x'].mean()], zoom_start=10)

# Create a marker cluster
marker_cluster = MarkerCluster().add_to(m)

# Add markers for each outage location
for idx, row in gdf.iterrows():
    folium.Marker([row['y'], row['x']], popup=row['Fault']).add_to(marker_cluster)

# Display the map
m

In [19]:
# Create a map centered at a location
m = folium.Map(location=[gdf['y'].mean(), gdf['x'].mean()], zoom_start=10)

# Create a MarkerCluster to group outage incidents by undertaking
marker_cluster = MarkerCluster().add_to(m)

# Get the total count of outages for each undertaking
undertaking_outage_counts = gdf['Undertaking'].value_counts().to_dict()

# Iterate through each row in the DataFrame
for index, row in gdf.iterrows():
    # Get the total count of outages for the undertaking of the current row
    total_outages = undertaking_outage_counts[row['Undertaking']]
    
    # Determine the color based on the total count of outages
    if total_outages >= 10:
        color = 'red'
    elif total_outages >= 5:
        color = 'orange'
    else:
        color = 'green'
    
    # Create a popup that includes the fault and the date
    popup = f"Fault: {row['Fault']}<br>Date: {row['Date']}"
    
    # Create a marker for each outage incident
    folium.Marker([row['y'], row['x']], popup=popup, icon=folium.Icon(color=color)).add_to(marker_cluster)

# Display the map
m

In [38]:
m.save("footprint.html")

In [20]:
# Remove rows with NaN values
gdf_cleaned = gdf.dropna()

# Or replace NaN values with a specific value
gdf_filled = gdf.fillna(0)  # Replace NaN with 0

In [21]:
import folium
from folium import Choropleth

# Create a map centered at a location
m = folium.Map(location=[gdf['y'].mean(), gdf['x'].mean()], zoom_start=10)

# Create a choropleth map based on outage data
choropleth = Choropleth(
    geo_data='path_to_geojson_file.geojson',  # GeoJSON file containing polygon boundaries
    data=gdf,  # DataFrame containing outage data
    columns=['Branch', 'Fault'],  # Columns for mapping
    key_on='feature.properties.Branch',  # Key in GeoJSON for mapping
    fill_color='YlOrRd',  # Color scale
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Outage Faults'
).add_to(m)

# Display the map
m

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [25]:
nan_values = gdf.isna().sum()
print(nan_values)


Branch         0
Date           0
Undertaking    0
Fault          0
Areas          0
y              0
x              0
geometry       0
dtype: int64


In [20]:
# Create a map centered at a location
m = folium.Map(location=[gdf['y'].mean(), gdf['x'].mean()], zoom_start=10)

# Create a MarkerCluster to group outage incidents by branch
marker_cluster = MarkerCluster().add_to(m)

# Iterate through each row in the DataFrame
for index, row in gdf.iterrows():
    # Define the color based on the number of outage incidents (you can modify this logic)
    if row['Fault'] == 'High':
        color = 'red'
    elif row['Fault'] == 'Medium':
        color = 'orange'
    else:
        color = 'green'
    
    # Create a marker for each outage incident
    folium.Marker([row['y'], row['x']], popup=row['Fault'], icon=folium.Icon(color=color)).add_to(marker_cluster)

# Display the map
m

In [ ]:
# Perfect, right now

import folium
from folium.plugins import MarkerCluster

# Create a map centered at a location
m = folium.Map(location=[gdf['y'].mean(), gdf['x'].mean()], zoom_start=10)

# Create a marker cluster
marker_cluster = MarkerCluster().add_to(m)

# Add markers for each outage location with popup clusters
for idx, row in gdf.iterrows():
    folium.Marker([row['y'], row['x']], popup=row['Fault']).add_to(marker_cluster)

# Display the map
m


In [21]:
# This is so perfect

import folium
from folium.plugins import TimestampedGeoJson

# Convert the DataFrame to GeoJSON with timestamp
features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [row['x'], row['y']]
        },
        'properties': {
            'time': str(row['Date']),
            'popup': row['Fault']
        }
    }
    for idx, row in gdf.iterrows()
]

# Create a map centered at a location
m = folium.Map(location=[gdf['y'].mean(), gdf['x'].mean()], zoom_start=10)

# Add the TimestampedGeoJson layer for time-series animation
TimestampedGeoJson(
    {'type': 'FeatureCollection', 'features': features},
    period='P1D',  # Time period
    add_last_point=True,
    auto_play=True
).add_to(m)

# Display the map
m
